# **KF on existing predictions**

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import json
import os

In [7]:
home_path = '/home/usuario/'

datasets_path = os.path.join(home_path, 'project_data', 'datasets')
new_path = os.path.join(home_path, 'project_data', 'new_datasets')

all_datasets = ['KITTITrackletsCars', 'KITTITrackletsCarsPersons', 'KITTITrackletsCarsHard', 'KITTITrackletsCarsPersonsHard']


In [2]:
def read_paths(filepath):
    """ Read path json file from filepath """
    with open(filepath) as file:
        name_cont = json.load(file)
    return name_cont

In [9]:
container_path = os.path.join(new_path, f"{all_datasets[0]}_path.json")
path_cont = read_paths(container_path)

In [17]:
# Load training jsons in memory
dataset_path = os.path.join(datasets_path, all_datasets[0])

for idn in path_cont['0_0']:
    meta_path = os.path.join(dataset_path, "meta", f"{idn}.json")
    pc1_path = os.path.join(dataset_path, "pointcloud1", f"{idn}.npy")
    pc2_path = os.path.join(dataset_path, "pointcloud2", f"{idn}.npy")
    
    

/home/usuario/project_data/datasets/KITTITrackletsCars/pc2/00000000.npy
/home/usuario/project_data/datasets/KITTITrackletsCars/pc2/00000001.npy
/home/usuario/project_data/datasets/KITTITrackletsCars/pc2/00000002.npy
/home/usuario/project_data/datasets/KITTITrackletsCars/pc2/00000003.npy
/home/usuario/project_data/datasets/KITTITrackletsCars/pc2/00000004.npy
/home/usuario/project_data/datasets/KITTITrackletsCars/pc2/00000005.npy
/home/usuario/project_data/datasets/KITTITrackletsCars/pc2/00000006.npy
/home/usuario/project_data/datasets/KITTITrackletsCars/pc2/00000007.npy
/home/usuario/project_data/datasets/KITTITrackletsCars/pc2/00000008.npy
/home/usuario/project_data/datasets/KITTITrackletsCars/pc2/00000009.npy
/home/usuario/project_data/datasets/KITTITrackletsCars/pc2/00000010.npy
/home/usuario/project_data/datasets/KITTITrackletsCars/pc2/00000011.npy
/home/usuario/project_data/datasets/KITTITrackletsCars/pc2/00000012.npy
/home/usuario/project_data/datasets/KITTITrackletsCars/pc2/00000

In [3]:
# Adapted from: https://github.com/xinshuoweng/AB3DMOT/blob/master/main.py
# Credits: Xinshuo Weng (https://github.com/xinshuoweng)

from filterpy.kalman import KalmanFilter

class KalmanMotionTracker(object):
  """
  This class represents the internal state of individual tracked objects observed as translation.
  """
  def __init__(self, position):
    """
    Initialises a tracker using initial position.

    KF Instance variables:
      x : ndarray (dim_x, 1), default = [0,0,0…0] filter state estimate
      P : ndarray (dim_x, dim_x), default eye(dim_x) covariance matrix
      Q : ndarray (dim_x, dim_x), default eye(dim_x) Process uncertainty/noise
      R : ndarray (dim_z, dim_z), default eye(dim_x) measurement uncertainty/noise
      H : ndarray (dim_z, dim_x) measurement function
      F : ndarray (dim_x, dim_x) state transistion matrix
      B : ndarray (dim_x, dim_u), default 0 control transition matrix
    """
    #define constant velocity model
    self.kf = KalmanFilter(dim_x=5, dim_z=3)   

    self.kf.x[:3] = position.reshape((3, 1))
    self.kf.P[3:,3:] *= 1000. #state uncertainty, give high uncertainty to the unobservable initial velocities, covariance matrix
    self.kf.P *= 10.
    self.kf.Q[3:,3:] *= 0.01


    self.kf.F = np.array([[1,0,0,1,0],      # state transition matrix
                          [0,1,0,0,1],
                          [0,0,1,0,0],
                          [0,0,0,1,0],  
                          [0,0,0,0,1]])
    
    self.kf.H = np.array([[1,0,0,0,0],      # measurement function,
                          [0,1,0,0,0],
                          [0,0,1,0,0]])
 

    # self.kf.R[0:,0:] *= 10.   # measurement uncertainty

    self.time_since_update = 0
    self.history = []
    self.hits = 1           # number of total hits including the first detection
    self.hit_streak = 1     # number of continuing hit considering the first detection
    self.first_continuing_hit = 1
    self.still_first = True
    self.age = 0

  def update(self, position): 
    """ 
    Updates the state vector with observed position.
    """
    self.time_since_update = 0
    self.history = []
    self.hits += 1
    self.hit_streak += 1          # number of continuing hit
    if self.still_first:
      self.first_continuing_hit += 1      # number of continuing hit in the fist time
    
    ######################### orientation correction NEEDED?
    if self.kf.x[3] >= np.pi: self.kf.x[3] -= np.pi * 2    # make the theta still in the range
    if self.kf.x[3] < -np.pi: self.kf.x[3] += np.pi * 2

    new_theta = position[3]
    if new_theta >= np.pi: new_theta -= np.pi * 2    # make the theta still in the range
    if new_theta < -np.pi: new_theta += np.pi * 2
    position[3] = new_theta

    predicted_theta = self.kf.x[3]
    if abs(new_theta - predicted_theta) > np.pi / 2.0 and abs(new_theta - predicted_theta) < np.pi * 3 / 2.0:     # if the angle of two theta is not acute angle
      self.kf.x[3] += np.pi       
      if self.kf.x[3] > np.pi: self.kf.x[3] -= np.pi * 2    # make the theta still in the range
      if self.kf.x[3] < -np.pi: self.kf.x[3] += np.pi * 2
      
    # now the angle is acute: < 90 or > 270, convert the case of > 270 to < 90
    if abs(new_theta - self.kf.x[3]) >= np.pi * 3 / 2.0:
      if new_theta > 0: self.kf.x[3] += np.pi * 2
      else: self.kf.x[3] -= np.pi * 2
    
    #########################

    self.kf.update(position)

    if self.kf.x[3] >= np.pi: self.kf.x[3] -= np.pi * 2    # make the theta still in the range
    if self.kf.x[3] < -np.pi: self.kf.x[3] += np.pi * 2

  def predict(self):       
    """
    Advances the state vector and returns the predicted bounding box estimate.
    """
    self.kf.predict()      
    if self.kf.x[3] >= np.pi: self.kf.x[3] -= np.pi * 2
    if self.kf.x[3] < -np.pi: self.kf.x[3] += np.pi * 2

    self.age += 1
    if(self.time_since_update>0):
          self.hit_streak = 0
          self.still_first = False
    self.time_since_update += 1
    self.history.append(self.kf.x)
    return self.history[-1]

  def get_state(self):
    """
    Returns the current bounding box estimate.
    """
    return self.kf.x[:7].reshape((7, ))